In [41]:
import tensorflow as tf
import tqdm
import cv2
import numpy as np
import matplotlib.pyplot as plt
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

new_x_train = x_train
new_x_test = x_test

_mean = np.mean(x_train)
_std = np.std(x_train)
print('Original image')
print(_mean, _std)
new_x_train = (new_x_train - _mean)/_std
new_x_test = (new_x_test - _mean)/_std
_mean = np.mean(new_x_train)
_std = np.std(new_x_train)
print('Standardization image')
print(_mean, _std)

model = tf.keras.models.load_model('MyCNN_ver01')
model.summary()

Num GPUs Available:  1
Original image
120.70756512369792 64.1500758911213
Standardization image
-2.5247951877342226e-17 1.0000000000000022
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 dropout (Dropout)           (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 dropout_1 (Dropout)         (None, 16, 16, 64)        0         
                                                 

In [42]:
model.evaluate(new_x_test, y_test, batch_size=32, verbose=2)

2023-01-05 21:43:11.437571: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 - 3s - loss: 0.7515 - accuracy: 0.7540 - 3s/epoch - 11ms/step


[0.7515486478805542, 0.7540000081062317]

In [81]:
for j in range(5):
    print(j * 10000, (j + 1) * 10000 - 1)


0 9999
10000 19999
20000 29999
30000 39999
40000 49999


In [87]:
dja = model(new_x_test)
for i in range(1000):
    print(tf.math.reduce_variance(dja[i]))


tf.Tensor(0.058919903, shape=(), dtype=float32)
tf.Tensor(0.07310659, shape=(), dtype=float32)
tf.Tensor(0.042664267, shape=(), dtype=float32)
tf.Tensor(0.013521693, shape=(), dtype=float32)
tf.Tensor(0.032878246, shape=(), dtype=float32)
tf.Tensor(0.05746443, shape=(), dtype=float32)
tf.Tensor(0.040925436, shape=(), dtype=float32)
tf.Tensor(0.04889339, shape=(), dtype=float32)
tf.Tensor(0.023986494, shape=(), dtype=float32)
tf.Tensor(0.019939061, shape=(), dtype=float32)
tf.Tensor(0.016467433, shape=(), dtype=float32)
tf.Tensor(0.08912854, shape=(), dtype=float32)
tf.Tensor(0.02167048, shape=(), dtype=float32)
tf.Tensor(0.08336818, shape=(), dtype=float32)
tf.Tensor(0.076063335, shape=(), dtype=float32)
tf.Tensor(0.056177486, shape=(), dtype=float32)
tf.Tensor(0.019347087, shape=(), dtype=float32)
tf.Tensor(0.077143654, shape=(), dtype=float32)
tf.Tensor(0.07472483, shape=(), dtype=float32)
tf.Tensor(0.089572124, shape=(), dtype=float32)
tf.Tensor(0.009366159, shape=(), dtype=float32)

In [82]:
# first, find not correct answer index
dja = np.ones(50000)
for j in range(5):
    _result = model(new_x_test[j * 10000 : (j + 1) * 10000 - 1])
    for i in tqdm.tqdm(range(10000)):
        if (tf.not_equal(tf.argmax(_result[i]), y_test[i])):
            # second, if(dja[i]==0) -> find var
            dja[i] = tf.math.reduce_variance(_result[i])
            
# result : 
#    if True -> 1
#    else -> this->var

  8%|▊         | 755/10000 [01:11<14:35, 10.56it/s]


KeyboardInterrupt: 